In [1]:
import scanpy as sc
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import doubletdetection
from scipy.stats import median_abs_deviation as mad
import numpy as np
import anndata as adt

In [2]:
import warnings 
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

In [3]:
path = '/home/ybeaumatin/Documents/Data processing pipeline/scP/Data/excel.data.treatment_allconditions_peptides.xlsx'

In [4]:
sheets = pd.read_excel(path, sheet_name=None, header = None)

In [5]:
cles = list(sheets.keys())

In [6]:
cles

['Nature fichier excel',
 'Mock3dpi_peptide',
 'Mock6dpi_peptide',
 'GFP.NEG3dpi_peptide',
 'GFP.NEG6dpi_peptide',
 'GFP.POS3dpi_peptide',
 'GFP.POS6dpi_peptide']

In [7]:
sheets['GFP.NEG3dpi_peptide']

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,Protein.Group,Protein.Ids,Protein.Names,Genes,First.Protein.Description,Proteotypic,Stripped.Sequence,Modified.Sequence,Precursor.Charge,Precursor.Id,...,H:\Perdu-Alloy Pauline\Coronavirus_HAE\GFP_neg...,H:\Perdu-Alloy Pauline\Coronavirus_HAE\GFP_neg...,H:\Perdu-Alloy Pauline\Coronavirus_HAE\GFP_neg...,H:\Perdu-Alloy Pauline\Coronavirus_HAE\GFP_neg...,H:\Perdu-Alloy Pauline\Coronavirus_HAE\GFP_neg...,H:\Perdu-Alloy Pauline\Coronavirus_HAE\GFP_neg...,H:\Perdu-Alloy Pauline\Coronavirus_HAE\GFP_neg...,H:\Perdu-Alloy Pauline\Coronavirus_HAE\GFP_neg...,H:\Perdu-Alloy Pauline\Coronavirus_HAE\GFP_neg...,H:\Perdu-Alloy Pauline\Coronavirus_HAE\GFP_neg...
1,P36578,P36578,RL4_HUMAN,RPL4,60S ribosomal protein L4,1,AAAAAAALQAK,AAAAAAALQAK,2,AAAAAAALQAK2,...,7206.45,4036.95,NaN,2451.48,2284.28,4423.05,NaN,3283.67,NaN,1833.87
2,Q8WUQ7,Q8WUQ7,CATIN_HUMAN,CACTIN,Cactin,1,AAAAALSQQQSLQER,AAAAALSQQQSLQER,2,AAAAALSQQQSLQER2,...,NaN,1664.99,NaN,2022.7,1990.73,1255.57,1801.82,NaN,1526.43,NaN
3,A6NIH7,A6NIH7,U119B_HUMAN,UNC119B,Protein unc-119 homolog B,1,AAAAASAAGPGGLVAGKEEK,AAAAASAAGPGGLVAGKEEK,3,AAAAASAAGPGGLVAGKEEK3,...,NaN,7072.06,NaN,15240.8,12272.6,6348.05,5831.31,7712.89,16084.9,NaN
4,Q9P258,Q9P258,RCC2_HUMAN,RCC2,Protein RCC2,1,AAAAAWEEPSSGNGTAR,AAAAAWEEPSSGNGTAR,2,AAAAAWEEPSSGNGTAR2,...,12170.8,13556.6,NaN,15588.8,17332.5,14010.9,11219.3,11596.3,14769.6,11929.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52480,P68104;Q5VTE0,P68104;Q5VTE0,EF1A1_HUMAN;EF1A3_HUMAN,EEF1A1;EEF1A1P5,Elongation factor 1-alpha 1,0,YYVTIIDAPGHRDFIK,YYVTIIDAPGHRDFIK,2,YYVTIIDAPGHRDFIK2,...,NaN,3441.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52481,Q8IYM9,Q8IYM9,TRI22_HUMAN,TRIM22,E3 ubiquitin-protein ligase TRIM22,1,YYWEVDVSGK,YYWEVDVSGK,2,YYWEVDVSGK2,...,NaN,3659.87,NaN,3323.5,2966.57,3794.59,NaN,NaN,4043.4,NaN
52482,P25311,P25311,ZA2G_HUMAN,AZGP1,Zinc-alpha-2-glycoprotein,1,YYYDGKDYIEFNK,YYYDGKDYIEFNK,3,YYYDGKDYIEFNK3,...,13318.4,3690.75,NaN,NaN,3973.51,NaN,12921.3,6014.19,NaN,13069.4
52483,Q8N183,Q8N183,NDUF2_HUMAN,NDUFAF2,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,1,YYYIPQYK,YYYIPQYK,2,YYYIPQYK2,...,NaN,NaN,NaN,2011.39,2799.84,6498.41,4499.34,5326.75,6386.72,NaN


In [68]:
adatas = []

for x in cles:
    sheet_part = sheets[x]
    var = sheet_part.iloc[1:, 0:10]  
    obs = pd.DataFrame(sheet_part.iloc[0, 10:])  
    data = sheet_part.iloc[1:, 10:].T

    adata = sc.AnnData(X=data, obs=obs, var=var) #create anndata
    
    #remodele the indexs
    #idx_var = pd.DataFrame(sheet_part.iloc[1:, 3]) 
    adata.obs.index = obs
    
    adata.obs.index = adata.obs.index.map(str)
    adata.obs.index = adata.obs.index.str.replace(r"[()']", "", regex=True)
    adata.obs.index = adata.obs.index.str.replace(";", "")
    adata.obs.index = adata.obs.index.str.replace(",", "")


    idx_var = pd.DataFrame(sheet_part.iloc[1:, 2])  # Index pour var (colonne cible)
    adata.var.index = [str(idx[0]) if isinstance(idx, tuple) else str(idx) for idx in idx_var.iloc[:, 0]]
    adata.var.index = [idx.replace("(", "").replace(")", "").replace("'", "").replace(";", "").replace(",", "") for idx in adata.var.index]
    
    adata.var.columns = sheet_part.iloc[0, 0:10]

    
    adata.obs = adata.obs.drop(columns=[0])
    
    adatas.append(adata)

In [69]:
adatas 

[AnnData object with n_obs × n_vars = 0 × 6
     var: 'FICHIER REGROUPANT RESULTATS PEPTIDES OBTENUS APRES INJECTIONS HAE COHORTE "1000 cellules"', nan, nan, nan, nan,
 AnnData object with n_obs × n_vars = 145 × 50450
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charge', 'Precursor.Id',
 AnnData object with n_obs × n_vars = 139 × 22563
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charge', 'Precursor.Id',
 AnnData object with n_obs × n_vars = 141 × 52484
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charge', 'Precursor.Id',
 AnnData object with n_obs × n_vars = 108 × 24151
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Ge

In [70]:
adatas[1].var

,Protein.Group,Protein.Ids,Protein.Names,Genes,First.Protein.Description,Proteotypic,Stripped.Sequence,Modified.Sequence,Precursor.Charge,Precursor.Id
RL4_HUMAN,P36578,P36578,RL4_HUMAN,RPL4,60S ribosomal protein L4,1,AAAAAAALQAK,AAAAAAALQAK,2,AAAAAAALQAK2
CATIN_HUMAN,Q8WUQ7,Q8WUQ7,CATIN_HUMAN,CACTIN,Cactin,1,AAAAALSQQQSLQER,AAAAALSQQQSLQER,2,AAAAALSQQQSLQER2
RCC2_HUMAN,Q9P258,Q9P258,RCC2_HUMAN,RCC2,Protein RCC2,1,AAAAAWEEPSSGNGTAR,AAAAAWEEPSSGNGTAR,2,AAAAAWEEPSSGNGTAR2
SPD2B_HUMAN,A1X283,A1X283,SPD2B_HUMAN,SH3PXD2B,SH3 and PX domain-containing protein 2B,1,AAAASVPNADGLK,AAAASVPNADGLK,2,AAAASVPNADGLK2
SPAG6_HUMAN,O75602,O75602,SPAG6_HUMAN,SPAG6,Sperm-associated antigen 6,1,AAAAWALGQIGR,AAAAWALGQIGR,2,AAAAWALGQIGR2
...,...,...,...,...,...,...,...,...,...,...
FIBB_HUMAN,P02675,P02675,FIBB_HUMAN,FGB,Fibrinogen beta chain,1,YYWGGQYTWDMAK,YYWGGQYTWDMAK,2,YYWGGQYTWDMAK2
DYH1_HUMAN,Q9P2D7,Q9P2D7,DYH1_HUMAN,DNAH1,Dynein axonemal heavy chain 1,1,YYWTNNDLYIR,YYWTNNDLYIR,2,YYWTNNDLYIR2
ZA2G_HUMAN,P25311,P25311,ZA2G_HUMAN,AZGP1,Zinc-alpha-2-glycoprotein,1,YYYDGKDYIEFNK,YYYDGKDYIEFNK,3,YYYDGKDYIEFNK3
NDUF2_HUMAN,Q8N183,Q8N183,NDUF2_HUMAN,NDUFAF2,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,1,YYYIPQYK,YYYIPQYK,2,YYYIPQYK2


In [71]:
adatas = adatas[1:]

In [72]:
adatas

[AnnData object with n_obs × n_vars = 145 × 50450
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charge', 'Precursor.Id',
 AnnData object with n_obs × n_vars = 139 × 22563
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charge', 'Precursor.Id',
 AnnData object with n_obs × n_vars = 141 × 52484
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charge', 'Precursor.Id',
 AnnData object with n_obs × n_vars = 108 × 24151
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charge', 'Precursor.Id',
 AnnData object with n_obs × n_vars = 13

In [73]:
def load_it(i, adata):
    adata.obs['Id'] = cles[i]
    return adata

In [74]:
adatas = [load_it(i,ad) for i,ad in enumerate (adatas)]

In [75]:
adatas

[AnnData object with n_obs × n_vars = 145 × 50450
     obs: 'Id'
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charge', 'Precursor.Id',
 AnnData object with n_obs × n_vars = 139 × 22563
     obs: 'Id'
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charge', 'Precursor.Id',
 AnnData object with n_obs × n_vars = 141 × 52484
     obs: 'Id'
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charge', 'Precursor.Id',
 AnnData object with n_obs × n_vars = 108 × 24151
     obs: 'Id'
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charg

In [76]:
from scipy.sparse import csr_matrix
for ad in adatas: 
    
    if isinstance(ad.X, np.ndarray):
         ad.X = ad.X.astype(np.float32)

    if isinstance(ad.X, csr_matrix):
         ad.X = csr_matrix(ad.X, dtype=np.float32)
    # Convertir toutes les colonnes de obs et var en chaînes
    ad.obs = ad.obs.astype(str)
    ad.var = ad.var.astype(str)

    # Convertir les index d'obs et var en chaînes
    ad.obs.index = ad.obs.index.map(str)
    ad.var.index = ad.var.index.map(str)


In [79]:
os.makedirs('/home/ybeaumatin/Documents/Data processing pipeline/scP/Data/peptidique/', exist_ok = True)

In [80]:
for ad in adatas:
    unique_id = ad.obs['Id'][0]
    file_path = os.path.join('/home/ybeaumatin/Documents/Data processing pipeline/scP/Data/peptidique/', f'adata_{unique_id}_peptides.h5ad')
    ad.write_h5ad(file_path)
    print(f"Saved: {file_path}")

Saved: /home/ybeaumatin/Documents/Data processing pipeline/scP/Data/peptidique/adata_Nature fichier excel_peptides.h5ad
Saved: /home/ybeaumatin/Documents/Data processing pipeline/scP/Data/peptidique/adata_Mock3dpi_peptide_peptides.h5ad
Saved: /home/ybeaumatin/Documents/Data processing pipeline/scP/Data/peptidique/adata_Mock6dpi_peptide_peptides.h5ad
Saved: /home/ybeaumatin/Documents/Data processing pipeline/scP/Data/peptidique/adata_GFP.NEG3dpi_peptide_peptides.h5ad
Saved: /home/ybeaumatin/Documents/Data processing pipeline/scP/Data/peptidique/adata_GFP.NEG6dpi_peptide_peptides.h5ad
Saved: /home/ybeaumatin/Documents/Data processing pipeline/scP/Data/peptidique/adata_GFP.POS3dpi_peptide_peptides.h5ad


In [81]:
adatas = []
data_dir = '/home/ybeaumatin/Documents/Data processing pipeline/scP/Data/peptidique/'

for filename in os.listdir(data_dir):
    if filename.endswith('.h5ad'):
        try:
            adata = sc.read_h5ad(os.path.join(data_dir, filename))
            adata.var_names_make_unique()
            adatas.append(adata)
        except OSError as e:
            print(f"Erreur lors de la lecture du fichier {filename}: {e}")

print(f"Nombre de fichiers h5ad chargés avec succès : {len(adatas)}")

Nombre de fichiers h5ad chargés avec succès : 6


In [82]:
adatas[0].X

array([[35456.   ,   694.633,  8529.66 , ...,   828.341,  1668.56 ,
         1897.9  ],
       [25116.   ,   771.252,  6389.11 , ...,       nan,  2788.05 ,
         1829.31 ],
       [24471.6  ,   495.583,  6071.05 , ...,       nan,  1461.18 ,
         1227.88 ],
       ...,
       [ 7605.46 ,       nan,  7271.89 , ...,       nan,       nan,
              nan],
       [      nan,  1362.87 ,  8917.92 , ...,  4420.88 ,       nan,
         4704.61 ],
       [      nan,       nan,  8309.17 , ...,  5113.33 ,       nan,
         2182.06 ]], dtype=float32)

In [83]:
adatas[0].var

,Protein.Group,Protein.Ids,Protein.Names,Genes,First.Protein.Description,Proteotypic,Stripped.Sequence,Modified.Sequence,Precursor.Charge,Precursor.Id
RL4_HUMAN,P36578,P36578,RL4_HUMAN,RPL4,60S ribosomal protein L4,1,AAAAAAALQAK,AAAAAAALQAK,2,AAAAAAALQAK2
CATIN_HUMAN,Q8WUQ7,Q8WUQ7,CATIN_HUMAN,CACTIN,Cactin,1,AAAAALSQQQSLQER,AAAAALSQQQSLQER,2,AAAAALSQQQSLQER2
RCC2_HUMAN,Q9P258,Q9P258,RCC2_HUMAN,RCC2,Protein RCC2,1,AAAAAWEEPSSGNGTAR,AAAAAWEEPSSGNGTAR,2,AAAAAWEEPSSGNGTAR2
SPD2B_HUMAN,A1X283,A1X283,SPD2B_HUMAN,SH3PXD2B,SH3 and PX domain-containing protein 2B,1,AAAASVPNADGLK,AAAASVPNADGLK,2,AAAASVPNADGLK2
SPAG6_HUMAN,O75602,O75602,SPAG6_HUMAN,SPAG6,Sperm-associated antigen 6,1,AAAAWALGQIGR,AAAAWALGQIGR,2,AAAAWALGQIGR2
...,...,...,...,...,...,...,...,...,...,...
FIBB_HUMAN-4,P02675,P02675,FIBB_HUMAN,FGB,Fibrinogen beta chain,1,YYWGGQYTWDMAK,YYWGGQYTWDMAK,2,YYWGGQYTWDMAK2
DYH1_HUMAN-18,Q9P2D7,Q9P2D7,DYH1_HUMAN,DNAH1,Dynein axonemal heavy chain 1,1,YYWTNNDLYIR,YYWTNNDLYIR,2,YYWTNNDLYIR2
ZA2G_HUMAN-8,P25311,P25311,ZA2G_HUMAN,AZGP1,Zinc-alpha-2-glycoprotein,1,YYYDGKDYIEFNK,YYYDGKDYIEFNK,3,YYYDGKDYIEFNK3
NDUF2_HUMAN-3,Q8N183,Q8N183,NDUF2_HUMAN,NDUFAF2,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,1,YYYIPQYK,YYYIPQYK,2,YYYIPQYK2


In [84]:
for ad in adatas:
    nbmiss = np.sum(np.isnan(ad.X))  # ad.X contient la matrice de données
    print(f"Nombre de valeurs NaN dans adata: {nbmiss}")

Nombre de valeurs NaN dans adata: 4254376
Nombre de valeurs NaN dans adata: 2398066
Nombre de valeurs NaN dans adata: 1520011
Nombre de valeurs NaN dans adata: 879677
Nombre de valeurs NaN dans adata: 2795025
Nombre de valeurs NaN dans adata: 1593406


In [85]:
for ad in adatas:
    nbmiss = np.sum(np.isnan(ad.X))/ np.sum(np.size(ad.X)) *100# ad.X contient la matrice de données
    print(f"Nombre de valeurs NaN dans adata: {nbmiss}")

Nombre de valeurs NaN dans adata: 58.15762960937767
Nombre de valeurs NaN dans adata: 49.00032284561849
Nombre de valeurs NaN dans adata: 48.465766676646716
Nombre de valeurs NaN dans adata: 33.72596334481971
Nombre de valeurs NaN dans adata: 37.76936273993128
Nombre de valeurs NaN dans adata: 54.21284329879861


In [86]:
peptidediff1 = set(adatas[2].var_names)
peptidediff1 = set(adatas[3].var_names)
allpeptidediff = (Genediff-genediff2)  ^(genediff2-Genediff)

In [87]:
len(allpeptidediff)

2606

In [88]:
for i,ad in enumerate(adatas):
    
    adatas[i] = ad[:, ~ad.var_names.isin(allpeptidediff)]

In [89]:
adatas

[View of AnnData object with n_obs × n_vars = 145 × 48662
     obs: 'Id'
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charge', 'Precursor.Id',
 View of AnnData object with n_obs × n_vars = 140 × 33781
     obs: 'Id'
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charge', 'Precursor.Id',
 View of AnnData object with n_obs × n_vars = 139 × 22210
     obs: 'Id'
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charge', 'Precursor.Id',
 View of AnnData object with n_obs × n_vars = 108 × 24055
     obs: 'Id'
     var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modi

In [90]:
for ad in adatas:
    nbmiss = np.sum(np.isnan(ad.X))/ np.sum(np.size(ad.X)) *100# ad.X contient la matrice de données
    print(f"Nombre de valeurs NaN dans adata: {nbmiss}")

Nombre de valeurs NaN dans adata: 57.46868972319972
Nombre de valeurs NaN dans adata: 48.31974440408175
Nombre de valeurs NaN dans adata: 48.045568947813386
Nombre de valeurs NaN dans adata: 33.67583546964133
Nombre de valeurs NaN dans adata: 36.761371232189674
Nombre de valeurs NaN dans adata: 53.696519501967465


In [91]:
adatas[1]

View of AnnData object with n_obs × n_vars = 140 × 33781
    obs: 'Id'
    var: 'Protein.Group', 'Protein.Ids', 'Protein.Names', 'Genes', 'First.Protein.Description', 'Proteotypic', 'Stripped.Sequence', 'Modified.Sequence', 'Precursor.Charge', 'Precursor.Id'

In [ ]:
all_genes = pd.Index(set().union(*[adata.var_names for adata in adatas]))
all_cells = pd.Index(set().union(*[adata.obs_names for adata in adatas]))

merged_X = pd.DataFrame(0, index=all_cells, columns=all_genes)

for adata in adatas:
    merged_X.loc[adata.obs_names, adata.var_names] = adata.X

merged_obs = pd.concat([adata.obs for adata in adatas], axis=0)

merged_var = pd.concat([adata.var for adata in adatas], axis=0).drop_duplicates()

merged_adata = ad.AnnData(X=merged_X, obs=merged_obs, var=merged_var)
